In [1]:
# !pip install adapter-transformers
# !pip install datasets
# !pip install evaluate
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os, sys
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"



import json
from transformers import AutoTokenizer, AdapterConfig, AutoAdapterModel, AutoConfig
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction, TrainerCallback
from transformers import AutoModelForSequenceClassification, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy, DataCollatorForLanguageModeling
from datasets import Dataset, DatasetDict

from tqdm import tqdm

import numpy as np
from datasets import concatenate_datasets, load_metric
import evaluate
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import random
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

set_seed(42)

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:
lang = "es"

df = pd.read_csv("../data/subtask_2/"+lang+"/train.tsv", sep='\t')
# df=df.drop(df.columns[0], axis=1)
df

,id,text,label
0,12786,"Sin embargo, los jóvenes son capaces de recono...",B
1,12361,¿Hay algo más que quieras compartir? ¿Algo sob...,B
2,1662,"El servicio de sala es bueno, rápido y amabilí...",B
3,14729,"Para concentrarse en el hablante, trata de des...",F
4,9312,Los responsables locales tendrán ahora que esp...,F
...,...,...,...
21930,11706,Escribe un correo electrónico a PI:EMAIL y cué...,A
21931,7997,"Los osos negros y grizzlies de Yellowstone, ta...",C
21932,11369,"El lenguaje es un recurso para comunicarse, pe...",A
21933,25259,"Dr, creo que las ostras son animales No, las o...",F


In [4]:
len(df.label.unique())

6

In [5]:
mapping = {
    "A": 0,
    "B": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5
}
df["label"] = df['label'].map(mapping)

In [6]:
from sklearn.model_selection import train_test_split

dataset_train_compl, dataset_test_compl = train_test_split(df, test_size=0.1, random_state=42)
dataset_train_compl, dataset_valid_compl = train_test_split(dataset_train_compl, test_size=0.1, random_state=42)

In [7]:

list_lm = [
#     "xlm-roberta-base", 
    "bert-base-multilingual-cased", 
#     "microsoft/deberta-v3-base",
#     "prajjwal1/bert-tiny",
#     "distilbert-base-cased",
#     "roberta-base-openai-detector",
#     "Hello-SimpleAI/chatgpt-detector-roberta"
]

t_metrics = {}

for language_model in list_lm:

    tokenizer = AutoTokenizer.from_pretrained(language_model)

    dataset_train = Dataset.from_pandas(dataset_train_compl)
    dataset_valid = Dataset.from_pandas(dataset_valid_compl)
    dataset_test = Dataset.from_pandas(dataset_test_compl)

    def encode_batch(batch):
        """Encodes a batch of input data using the model tokenizer."""
        return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")


    dataset_train = dataset_train.rename_column("label", "labels")
    dataset_train = dataset_train.map(encode_batch, batched=True)
    dataset_train.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])

    dataset_valid = dataset_valid.rename_column("label", "labels")
    dataset_valid = dataset_valid.map(encode_batch, batched=True)
    dataset_valid.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])

    dataset_test = dataset_test.map(encode_batch, batched=True)
    dataset_test = dataset_test.rename_column("label", "labels")
    dataset_test.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
    
    model = AutoModelForSequenceClassification.from_pretrained(language_model, num_labels=len(df.label.unique()), ignore_mismatched_sizes=True)
  



    def compute_metrics(pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
        acc = accuracy_score(labels, preds)
        return {
            'accuracy': acc,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }

    # data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
    early_stop = EarlyStoppingCallback(3)

    training_args = TrainingArguments(
        learning_rate=1e-6,
        num_train_epochs=10,
        seed = 42,
        output_dir="./training_output_mbert_subtask2",
        # label_names=["generated", "human"]
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        dataloader_num_workers=32,
        logging_steps=100,
        save_total_limit = 2,
        overwrite_output_dir=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        report_to='tensorboard',
        metric_for_best_model='f1'
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_valid,
        compute_metrics=compute_metrics,
        callbacks = [early_stop]
    )

    trainer.train()
    
    t_metrics[language_model] = trainer.evaluate(dataset_test)
    print(t_metrics)
pd.DataFrame([t_metrics]).to_csv("./result/_subtask2_"+str(lang)+".csv")

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining mo

Epoch,Training Loss,Validation Loss



KeyboardInterrupt



## Modelling

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(language_model, num_labels=len(df.label.unique()), ignore_mismatched_sizes=True)
  
# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }

# # data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
# early_stop = EarlyStoppingCallback(3)

# training_args = TrainingArguments(
#     learning_rate=1e-6,
#     num_train_epochs=10,
#     seed = 42,
#     output_dir="./training_output_mbert_subtask2",
#     # label_names=["generated", "human"]
#     per_device_train_batch_size=64,
#     per_device_eval_batch_size=64,
#     dataloader_num_workers=32,
#     logging_steps=100,
#     save_total_limit = 2,
#     overwrite_output_dir=True,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     report_to='tensorboard',
#     metric_for_best_model='f1'
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset_train,
#     eval_dataset=dataset_valid,
#     compute_metrics=compute_metrics,
#     callbacks = [early_stop]
# )

# trainer.train()

In [ ]:
# t_metrics = trainer.evaluate(dataset_test)
# t_metrics

In [ ]:
# pd.DataFrame([t_metrics])